<a href="https://colab.research.google.com/github/mbhargava4/CS-539-Final-Project-Equation-Solver/blob/main/Convert_inkMLtoPNG_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
import xml.etree.ElementTree as ET
import os
from tqdm import tqdm
import cv2
!pip install split-folders
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [26]:
path = '/content/drive/MyDrive/ECE_class/CS 539 Group /data_inkml'
def seed_everything(seed):

    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


SEED = 999
seed_everything(SEED)

In [27]:
def get_traces_data(inkml_file_abs_path):


    traces_data = []

    tree = ET.parse(inkml_file_abs_path)
    root = tree.getroot()
    doc_namespace = "{http://www.w3.org/2003/InkML}"
    traces_all = [{'id': trace_tag.get('id'),
    					'coords': [[round(float(axis_coord)) if float(axis_coord).is_integer() else round(float(axis_coord) * 10000) \
    									for axis_coord in coord[1:].split(' ')] if coord.startswith(' ') \
    								else [round(float(axis_coord)) if float(axis_coord).is_integer() else round(float(axis_coord) * 10000) \
    									for axis_coord in coord.split(' ')] \
    							for coord in (trace_tag.text).replace('\n', '').split(',')]} \
    							for trace_tag in root.findall(doc_namespace + 'trace')]

    traces_all.sort(key=lambda trace_dict: int(trace_dict['id']))
    traceGroupWrapper = root.find(doc_namespace + 'traceGroup')

    if traceGroupWrapper is not None:
        for traceGroup in traceGroupWrapper.findall(doc_namespace + 'traceGroup'):

            label = traceGroup.find(doc_namespace + 'annotation').text

            traces_curr = []
            for traceView in traceGroup.findall(doc_namespace + 'traceView'):

                traceDataRef = int(traceView.get('traceDataRef'))

                single_trace = traces_all[traceDataRef]['coords']
                traces_curr.append(single_trace)

            traces_data.append({'label': label, 'trace_group': traces_curr})

    else:
        [traces_data.append({'trace_group': [trace['coords']]}) for trace in traces_all]

    return traces_data

In [28]:
def inkml2img(input_path, output_path):
    traces = get_traces_data(input_path)
    path_parts = input_path.split('/')
    filename = path_parts[-1].split('.')[0]
    for i, elem in enumerate(traces):
        plt.gca().invert_yaxis()
        plt.gca().set_aspect('equal', adjustable='box')
        plt.axis('off')
        plt.plot([0], [0])
        ls = elem.get('trace_group', [])  # Use .get() to safely retrieve 'trace_group'
        for subls in ls:
            data = np.array(subls)
            x, y = zip(*data)
            plt.plot(x, y, linewidth=2, c='black')

        os.makedirs(output_path, exist_ok=True)
        filepath = os.path.join(output_path, f"{filename}_{i}.png")
        if not os.path.exists(filepath):
            plt.savefig(filepath, bbox_inches='tight', pad_inches=0, dpi=100)
            plt.clf() # Clear current figure

In [29]:
images = os.listdir(os.getcwd() + '/drive/MyDrive/ECE_class/CS 539 Group /data_inkml/CROHME_test_2011')
for image in images:
    inkml2img('/content/drive/MyDrive/ECE_class/CS 539 Group /data_inkml/CROHME_test_2011/'+image, '/content/drive/MyDrive/ECE_class/CS 539 Group /processed_data_test')

<Figure size 640x480 with 0 Axes>